# 2. Microsoft Foundry Agent - 작업 단계별 표시

이 노트북에서는 **Microsoft Foundry의 Agent Service**를 사용하여 AI 에이전트의 작업 과정을 단계별로 표시하는 방법을 보여줍니다.

## 주요 내용
- 🤖 **Microsoft Foundry Agent**: 에이전트 생성 및 실행
- 📋 **Run Steps**: 에이전트 실행 단계 추적 API
- 🔧 **도구 호출 표시**: Code Interpreter, File Search, Function 등
- 📊 **실시간 상태**: 스트리밍 이벤트로 진행 상황 표시

> 📖 참고: [Microsoft Foundry Agent Service 문서](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/)

---

## 2.1 환경 설정 및 라이브러리 임포트

In [1]:
# 필요한 패키지 설치
%pip install --quiet azure-ai-projects azure-identity python-dotenv


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
import time
import json
from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown, clear_output

# Azure AI SDK
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

from azure.ai.projects.models import PromptAgentDefinition
from azure.ai.projects.models import CodeInterpreterTool
from azure.ai.projects.models import WebSearchPreviewTool, ApproximateLocation

# 환경 변수 로드
load_dotenv()

print("✅ 라이브러리 로드 완료!")

✅ 라이브러리 로드 완료!


In [10]:
# Microsoft Foundry Project 클라이언트 설정
PROJECT_ENDPOINT = os.getenv("AZURE_AI_PROJECT_ENDPOINT", "")

if PROJECT_ENDPOINT:
    credential = DefaultAzureCredential()
    client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)
    print("✅ Microsoft Foundry Project 클라이언트 연결 완료!")
    print(f"   엔드포인트: {PROJECT_ENDPOINT}")
else:
    client = None
    print("⚠️ AZURE_AI_PROJECT_ENDPOINT가 설정되지 않았습니다.")
    print("   .env 파일에 엔드포인트를 추가해주세요.")
    print("   예: https://<project-name>.services.ai.azure.com/api/projects/<project-id>")

✅ Microsoft Foundry Project 클라이언트 연결 완료!
   엔드포인트: https://ms-foundry-1023.services.ai.azure.com/api/projects/ms-foundry


## 2.2 WebSearch 에이전트 생성 및 테스트

In [ ]:
# 2.2 WebSearchAgent 생성
WEB_SEARCH_INSTRUCTIONS = """너는 Tools에 등록된 Web search 기반으로 답변하는 에이전트입니다.

중요 규칙:
1. 최신 정보가 필요한 질문에는 반드시 웹 검색을 사용하세요
2. 검색 결과를 기반으로 정확하고 최신의 정보를 제공하세요
3. 답변 시 출처 URL을 포함하세요
4. 여러 출처의 정보를 종합하여 균형잡힌 답변을 제공하세요
5. 검색 결과가 불충분하면 추가 검색을 수행하세요"""

try:
    definition = PromptAgentDefinition(
        model="gpt-4.1",
        instructions=WEB_SEARCH_INSTRUCTIONS,
        tools=[
          WebSearchPreviewTool()
        #   WebSearchPreviewTool(
        #     user_location=ApproximateLocation(country="GB", city="London", region="London")
        #   )
        ]
    )
    
    agent = client.agents.create(
        name="WebSearchAgent",
        definition=definition
    )
    
    print(f"✅ WebSearchAgent 생성 완료!")
    print(f"   ID: {agent.id}")
    print(f"   Name: {agent.name}")
    print(f"   Model: {definition.model}")
    print(f"   Tools: Web Search")
    
except Exception as e:
    print(f"⚠️ 에이전트 생성 실패: {e}")
    print("\n💡 해결 방법:")
    print("   1. 'gpt-4.1' 모델이 배포되었는지 확인")
    print("   2. Web Search가 프로젝트에서 활성화되었는지 확인")

✅ WebSearchAgent 생성 완료!
   ID: WebSearchAgent
   Name: WebSearchAgent
⚠️ 에이전트 생성 실패: 'AgentDetails' object has no attribute 'version'

💡 해결 방법:
   1. 'gpt-4.1' 모델이 배포되었는지 확인
   2. Web Search가 프로젝트에서 활성화되었는지 확인


### 함수 호출 테스트

In [ ]:
try:
    print("⏳ WebSearchAgent 실행 중 (웹 검색 중...)")
    print(f"📍 Agent Name: {agent.name}")
    
    # SDK v2 - project client를 통해 OpenAI client 획듍
    openai_client = client.get_openai_client()
    
    # Conversation 생성
    conversation = openai_client.conversations.create()
    print(f"💬 Conversation ID: {conversation.id}")
    
    # Responses API 호출
    response = openai_client.responses.create(
        conversation=conversation.id,
        input="오늘 서울 날씨는 어떤가요?",
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    )
    
    print("\n" + "=" * 80)
    print("🌐 WebSearchAgent 응답:")
    print("=" * 80)
    print(response.output_text)
    
    # Conversation 정리
    openai_client.conversations.delete(conversation_id=conversation.id)
    print("\n✅ 웹 검색 도구가 실시간 정보를 가져왔습니다!")
    
except NameError:
    print("⚠️ agent 또는 client가 정의되지 않았습니다.")
    print("💡 위의 환경 설정 및 WebSearchAgent 생성 셀을 먼저 실행하세요.")
    
except Exception as e:
    print(f"⚠️ 테스트 실패: {e}")
    import traceback
    print(f"\n상세 오류:\n{traceback.format_exc()}")

⏳ WebSearchAgent 실행 중 (웹 검색 중...)
📍 Agent Name: WebSearchAgent
💬 Conversation ID: conv_a4ed7f6c2e51b37d005WIWQbUId7CKcy3aUP9OXZiFAgPfpgCL

🌐 WebSearchAgent 응답:
2026년 2월 1일 오늘 서울의 날씨는 아침에 영하 7도 내외로 상당히 춥고, 낮에는 2도 정도까지 오르며 조금 풀릴 전망입니다. 전국적으로 영하권 아침 기온과 낮 동안의 비교적 온화한 날씨가 예상되며, 밤에는 눈 소식이 있을 수 있습니다. 바람도 조금 불어서 체감 추위가 더 심할 수 있으니 외출 시 옷차림에 신경 쓰는 것이 좋습니다[국민일보](https://www.kmib.co.kr/article/view.asp?arcid=0029331628), [HikersBay](https://hikersbay.com/climate/february/southkorea/seoul?lang=ko).

요약:
- 아침 최저기온: -7°C
- 낮 최고기온: +2°C
- 체감 추위 강함, 밤부터 일부 눈 가능성

추가로, 2월 전체는 평년과 비슷하거나 약간 따뜻한 흐름이지만, 초순은 추운 날씨가 반복될 수 있으니 결빙 및 체감 한파에도 주의가 필요합니다[기상청 전망](https://admystory.tistory.com/entry/2026%EB%85%84-2%EC%9B%94-%EB%82%A0%EC%94%A8-%EC%A0%84%EB%A7%9D-%EC%A3%BC%EC%B0%A8%EB%B3%84-%EA%B8%B0%EC%98%A8-%EB%B3%80%ED%99%94%EC%99%80-%EB%88%88%C2%B7%EB%B9%84-%EC%83%81%EC%84%B8-%EC%98%88%EB%B3%B4%EC%B5%9C%EC%8B%A0-%EC%9E%90%EB%A3%8C-%EC%97%85%EB%8D%B0%EC%9D%B4%ED%8A%B8).

날씨 변화가 크니 당일 실시간 예보도 확인해 주세요.

✅ 웹 검색 도구가 실시

### 스트리밍 방식 함수 호출 테스트

In [22]:
# Conversation 생성
conversation = openai_client.conversations.create()
print(f"💬 Conversation ID: {conversation.id}")
print()
    
stream_response = openai_client.responses.create(
    stream=True,
    conversation=conversation.id,
    input="오늘 서울 날씨는 어떤가요?",
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
)

print("🤖 응답: ", end="", flush=True)

for event in stream_response:
    if event.type == "response.created":
        pass  # 시작 시 별도 출력 불필요
    elif event.type == "response.output_text.delta":
        # 각 delta를 연결하여 출력 (줄바꿈 없이)
        print(event.delta, end="", flush=True)
    elif event.type == "response.output_item.done":
        if event.item.type == "message":
            item = event.item
            if item.content[-1].type == "output_text":
                text_content = item.content[-1]
                # URL 인용 출력
                if text_content.annotations:
                    print("\n\n📚 출처:")
                    for annotation in text_content.annotations:
                        if annotation.type == "url_citation":
                            print(f"   🔗 {annotation.url}")
    elif event.type == "response.completed":
        print("\n\n✅ 응답 완료!")

💬 Conversation ID: conv_793f03c049620e5d00cHlkxD2z5Lo1K3CyjMNwyHFIqPHQP0wq

🤖 응답: 2026년 2월 1일 서울의 날씨는 아침에 영하권의 추운 날씨가 예상되며, 서울의 아침 최저기온은 -7도, 낮 최고기온은 2도 정도로 전망됩니다. 전국적으로 낮에는 기온이 조금 오르면서 활동하기가 수월해질 것으로 보이지만, 하루 전체적으로는 겨울답게 찬 공기가 남아있습니다. 밤부터는 전국적으로 눈이 예보되어 있으니 이동 계획에 참고하시면 좋겠습니다[국민일보](https://www.kmib.co.kr/article/view.asp?arcid=0029331628), [xdnpq.tistory](https://xdnpq.tistory.com/entry/weather-2026-02-01).

요약
- 서울 아침 최저 -7℃
- 서울 낮 최고 2℃
- 전국적으로 밤부터 눈 예보
- 낮 동안은 비교적 포근하지만 체감은 쌀쌀함

추위와 눈 예보로 인한 결빙 위험, 강수 타이밍을 고려해 일정을 계획하시기 바랍니다. 최신 정보는 외출 전에도 다시 확인하시는 것이 안전합니다.

📚 출처:
   🔗 https://www.kmib.co.kr/article/view.asp?arcid=0029331628
   🔗 https://xdnpq.tistory.com/entry/weather-2026-02-01


✅ 응답 완료!


## 2.3 진행 상황 표시 함수

에이전트 실행 단계를 시각적으로 표시하는 간단한 헬퍼 함수들입니다.

In [24]:
# 단계 유형별 아이콘
STEP_ICONS = {
    "thread_created": "📝",
    "message_added": "💬",
    "thinking": "🧠",
    "tool_calls": "🔧",
    "code_interpreter": "💻",
    "file_search": "📂",
    "function_call": "⚡",
    "message_creation": "✍️",
    "completed": "✅",
    "error": "❌",
}


def display_steps(steps: list, title: str = "Microsoft Foundry Agent 실행 중", is_complete: bool = False):
    """
    단계 목록을 HTML로 표시
    
    Args:
        steps: [{"type": str, "text": str, "status": str, "time": float}, ...]
        title: 표시할 제목
        is_complete: 완료 여부
    """
    if is_complete:
        bg_color = "rgba(76, 175, 80, 0.15)"
        border_color = "rgba(76, 175, 80, 0.4)"
        title_color = "#81c784"
    else:
        bg_color = "rgba(33, 150, 243, 0.15)"
        border_color = "rgba(33, 150, 243, 0.4)"
        title_color = "#64b5f6"
    
    html = f"""
    <div style="font-family: 'Segoe UI', sans-serif; padding: 15px; background: {bg_color}; 
                border: 1px solid {border_color}; border-radius: 10px; max-width: 650px;">
        <div style="font-weight: bold; color: {title_color}; margin-bottom: 12px; font-size: 1.1em;">
            🤖 {title}
        </div>
        <div style="border-left: 3px solid #0078d4; padding-left: 12px;">
    """
    
    for step in steps:
        step_type = step.get("type", "")
        icon = STEP_ICONS.get(step_type, "📌")
        status = step.get("status", "running")
        
        # 상태별 색상
        if status == "completed":
            status_icon = "✅"
            text_color = "#81c784"
        elif status == "running":
            status_icon = "⏳"
            text_color = "#64b5f6"
        elif status == "error":
            status_icon = "❌"
            text_color = "#ef5350"
        else:
            status_icon = "⚪"
            text_color = "#9e9e9e"
        
        time_str = f"+{step.get('time', 0):.1f}s" if step.get('time') else ""
        
        html += f"""
        <div style="padding: 6px 0; color: {text_color};">
            {status_icon} {icon} {step.get("text", "")}
            <span style="color: #bdbdbd; font-size: 0.85em; float: right;">{time_str}</span>
        </div>
        """
    
    html += "</div></div>"
    display(HTML(html))


def display_final_response(response_text: str, total_steps: int, total_time: float):
    """최종 응답을 표시"""
    html = f"""
    <div style="font-family: 'Segoe UI', sans-serif; padding: 15px; background: rgba(76, 175, 80, 0.15); 
                border: 1px solid rgba(76, 175, 80, 0.4); border-radius: 10px; max-width: 650px; margin-top: 10px;">
        <div style="font-weight: bold; color: #81c784; margin-bottom: 8px;">
            ✅ 작업 완료!
        </div>
        <div style="color: #e0e0e0; font-size: 0.9em;">
            📊 총 {total_steps}개 단계 | ⏱️ {total_time:.2f}초
        </div>
    </div>
    """
    display(HTML(html))
    display(Markdown(response_text))


print("✅ 진행 상황 표시 함수 정의 완료!")

✅ 진행 상황 표시 함수 정의 완료!


## 2.4 WebSearch Agent 실행 함수

앞서 생성한 WebSearchAgent를 활용하여 진행 상황을 표시하는 함수들입니다.

### Responses API 구조

```
openai_client.responses.create()
 └── Stream Events
      ├── response.created: 응답 시작
      ├── response.output_text.delta: 텍스트 토큰 스트리밍
      ├── response.output_item.done: 항목 완료 (annotations 포함)
      └── response.completed: 응답 완료
```

In [28]:
def run_websearch_agent(user_message: str, show_sources: bool = True):
    """
    WebSearchAgent를 실행하고 응답을 반환
    
    Args:
        user_message: 사용자 질문
        show_sources: 출처 URL 표시 여부
    
    Returns:
        응답 텍스트
    """
    if not client or not agent:
        print("⚠️ client 또는 agent가 설정되지 않았습니다.")
        return None
    
    steps = []
    start_time = time.time()
    
    def add_step(step_type: str, text: str, status: str = "running"):
        steps.append({
            "type": step_type,
            "text": text,
            "status": status,
            "time": time.time() - start_time
        })
        clear_output(wait=True)
        display_steps(steps, title="WebSearch Agent 실행 중")
    
    def complete_last_step(new_text: str = None):
        if steps:
            steps[-1]["status"] = "completed"
            if new_text:
                steps[-1]["text"] = new_text
            clear_output(wait=True)
            display_steps(steps, title="WebSearch Agent 실행 중")
    
    try:
        # 1. OpenAI Client 획득
        add_step("thinking", "OpenAI Client 준비 중...")
        openai_client = client.get_openai_client()
        complete_last_step("OpenAI Client 준비 완료")
        
        # 2. Conversation 생성
        add_step("thread_created", "Conversation 생성 중...")
        conversation = openai_client.conversations.create()
        complete_last_step(f"Conversation 생성 완료 (ID: {conversation.id[:12]}...)")
        
        # 3. 웹 검색 시작
        add_step("tool_calls", "🌐 웹 검색 중...")
        
        response = openai_client.responses.create(
            conversation=conversation.id,
            input=user_message,
            extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
        )
        complete_last_step("웹 검색 완료")
        
        # 4. 응답 생성 완료
        add_step("message_creation", "응답 생성 중...")
        complete_last_step("응답 생성 완료")
        
        add_step("completed", "작업 완료", status="completed")
        
        # 결과 표시
        total_time = time.time() - start_time
        clear_output(wait=True)
        display_steps(steps, title="WebSearch Agent 완료", is_complete=True)
        display_final_response(response.output_text, len(steps), total_time)
        
        # 출처 표시
        if show_sources:
            for item in response.output:
                if item.type == "message" and item.content:
                    for content in item.content:
                        if content.type == "output_text" and content.annotations:
                            print("\n📚 출처:")
                            for annotation in content.annotations:
                                if annotation.type == "url_citation":
                                    print(f"   🔗 {annotation.url}")
        
        # Conversation 정리
        openai_client.conversations.delete(conversation_id=conversation.id)
        
        return response.output_text
        
    except Exception as e:
        add_step("error", f"오류 발생: {str(e)}", status="error")
        raise


print("✅ run_websearch_agent() 함수 정의 완료!")
print("   사용법: run_websearch_agent('오늘 서울 날씨는?')")

✅ run_websearch_agent() 함수 정의 완료!
   사용법: run_websearch_agent('오늘 서울 날씨는?')


In [29]:
# 실행 예제
response = run_websearch_agent("최근 AI 에이전트 트렌드에 대해 알려주세요.")

최근 AI 에이전트 트렌드는 다음과 같이 요약할 수 있습니다.

### 1. **AI 에이전트 시장 급성장**
- 기업이 업무 프로세스 혁신, 디지털 전환 수요 확대에 힘입어 AI 에이전트 시장이 2024년 기준 약 300~400억 달러에 도달, 2030년에는 1,500~2,000억 달러 이상까지 성장할 것으로 예측되고 있습니다.
- 기업의 63%가 AI 에이전트를 향후 1년 내 우선순위로 고려하고 있으며, 투자와 실무 도입이 급증하고 있습니다. 특히, 대형 테크 기업(구글, 마이크로소프트, 오픈AI 등)은 범용 에이전트 개발, 기업 맞춤 AI 도입에 경쟁적으로 뛰어들고 있습니다 [[KT 클라우드](https://tech.ktcloud.com/entry/2025-12-ktcloud-ai-agents-%EC%82%B0%EC%97%85-%ED%8C%A8%EB%9F%AC%EB%8B%A4%EC%9E%84)] [[CB Insight/브런치](https://brunch.co.kr/@bb02a3bf8a014f2/9)].

### 2. **AI 에이전트의 기능 및 특징**
- 기존 LLM(예: 챗GPT)이 단순 대화와 정보 제공에 집중했던 것과 달리, AI 에이전트는 사용자의 목표 파악, 자율적 계획 수립, 복잡한 업무 실행, 도구 및 외부 시스템 활용 등 능동적 “행동”에 초점을 맞추고 있습니다.
- 업무 비서, 데이터 분석 자동화, 소프트웨어 코딩 보조, 헬스케어·금융·법률 등 분야 특화 AI 에이전트가 등장했습니다 [[SK AX](https://www.skax.co.kr/insight/trend/3094)].

### 3. **산업·기업에서의 본격 도입과 확산**
- 실제 기업 현장에서는 고객 상담 자동화, 영업/마케팅 워크플로우 관리, 품질/재고 관리, 자동 번역·문서 작성, 진단 및 환자 모니터링(헬스케어) 등 다양한 업무에 AI 에이전트 적용이 확산되고 있습니다.
- 트렌드는 “범용”에서 “산업 특화(Verticalized)”로 발전하고 있습니다. 예를 들어 의료, 금융, 법률 등 규제가 강한 분야에서는 전문화된 에이전트가 성장 중입니다 [[CB Insight/브런치](https://brunch.co.kr/@bb02a3bf8a014f2/9)].

### 4. **AI 에이전트 기술/플랫폼 인프라와 이슈**
- 개발을 위한 인프라(노코드/로우코드 툴, 데이터 연결 솔루션, 평가 시스템 등) 시장이 커지고 있습니다.
- 생성형 AI 모델 상향 평준화(GPT-4o, Gemini, Claude 등)로 AI 에이전트의 자율성과 복합 멀티태스킹 능력이 강화되고, 실제 업무에서의 오류·보안·신뢰성 이슈를 해결하는 게 차세대 경쟁 포인트가 되고 있습니다 [[Superb AI](https://blog-ko.superb-ai.com/ai-trends-2024-review/)].

### 5. **향후 전망 및 과제**
- 2025~2027년 사이 AI 에이전트가 모든 정보 노동에 필수적 역할을 할 것이라는 낙관론과 함께, 절반 가까운 프로젝트가 조기 실패할 비관론(도입 비용/ROI 불확실성 및 과대광고 논란)도 제기되고 있습니다 [[삼일PwC](https://www.pwc.com/kr/ko/insights/issue-brief/samilpwc_ai-agent-in-action.pdf)].
- 성공적 도입을 위해서는 “Human-in-the-loop(인간 개입형 시스템)”을 병행하고 신뢰성, 책임성, 데이터 품질, 보안 관리 등 현실적 문제를 해결해 나가는 것이 중요합니다.

**정리**
2024~2025년 AI 트렌드는 대규모 언어모델의 한계를 넘어 실제로 ‘일하는 AI’의 시대, 즉 “능동적 행동”과 “업무 자동화”를 이끄는 AI 에이전트가 시장과 사회의 중심 키워드로 부상하고 있습니다.

**더 자세한 내용은 다음 자료를 참고하세요:**  
- [삼정KPMG 보고서: AI 에이전트 혁신](https://assets.kpmg.com/content/dam/kpmg/kr/pdf/2025/issue-monitor/%EC%82%BC%EC%A0%95KPMG-AI%EC%97%90%EC%9D%B4%EC%A0%84%ED%8A%B8-20250908.pdf)
- [CB Insight 기반 정리](https://brunch.co.kr/@bb02a3bf8a014f2/9)
- [SK AX 트렌드 리포트](https://www.skax.co.kr/insight/trend/3094)


📚 출처:
   🔗 https://assets.kpmg.com/content/dam/kpmg/kr/pdf/2025/issue-monitor/%EC%82%BC%EC%A0%95KPMG-AI%EC%97%90%EC%9D%B4%EC%A0%84%ED%8A%B8-20250908.pdf
   🔗 https://brunch.co.kr/@bb02a3bf8a014f2/9
   🔗 https://tech.ktcloud.com/entry/2025-12-ktcloud-ai-agents-%EC%82%B0%EC%97%85-%ED%8C%A8%EB%9F%AC%EB%8B%A4%EC%9E%84
   🔗 https://www.pwc.com/kr/ko/insights/issue-brief/samilpwc_ai-agent-in-action.pdf
   🔗 https://blog-ko.superb-ai.com/ai-trends-2024-review/
   🔗 https://www.skax.co.kr/insight/trend/3094


## 2.5 스트리밍 + 상세 단계 표시

`stream=True` 옵션으로 실시간 이벤트를 수신하며, 각 에이전트 작업 단계를 시각적으로 표시합니다.

- 참고
  - https://platform.openai.com/docs/api-reference/responses-streaming
  - https://learn.microsoft.com/ko-kr/azure/ai-foundry/agents/how-to/tools/web-search?view=foundry&pivots=python
  
### 웹 검색 에이전트 작업 단계

| 단계 | 이벤트 | 설명 |
|------|--------|------|
| 1️⃣ 준비 | - | OpenAI Client 및 Conversation 생성 |
| 2️⃣ 웹 검색 시작 | `response.created` | 에이전트가 웹 검색 도구 호출 |
| 3️⃣ 검색 결과 수신 | `response.output_item.done (function_call)` | 웹 검색 결과 반환 |
| 4️⃣ 응답 생성 | `response.output_text.delta` | 검색 결과 기반 응답 작성 |
| 5️⃣ 완료 | `response.completed` | 모든 작업 완료 |

In [32]:
def run_websearch_agent_with_steps(user_message: str, show_sources: bool = True):
    """
    스트리밍 API로 WebSearchAgent 실행하며 각 단계를 상세히 표시
    
    Args:
        user_message: 사용자 질문
        show_sources: 출처 URL 표시 여부
    
    Returns:
        전체 응답 텍스트
    """
    if not client or not agent:
        print("⚠️ client 또는 agent가 설정되지 않았습니다.")
        return None
    
    steps = []
    start_time = time.time()
    full_response = ""
    sources = []
    web_search_started = False
    response_started = False
    
    def add_step(step_type: str, text: str, status: str = "running"):
        steps.append({
            "type": step_type,
            "text": text,
            "status": status,
            "time": time.time() - start_time
        })
        clear_output(wait=True)
        display_steps(steps, title="🌐 WebSearch Agent 실행 중")
    
    def complete_last_step(new_text: str = None):
        if steps:
            steps[-1]["status"] = "completed"
            if new_text:
                steps[-1]["text"] = new_text
            clear_output(wait=True)
            display_steps(steps, title="🌐 WebSearch Agent 실행 중")
    
    try:
        # 1. 준비 단계
        add_step("thinking", "OpenAI Client 준비 중...")
        openai_client = client.get_openai_client()
        complete_last_step("OpenAI Client 준비 완료")
        
        # 2. Conversation 생성
        add_step("thread_created", "Conversation 생성 중...")
        conversation = openai_client.conversations.create()
        complete_last_step(f"Conversation 생성 완료")
        
        # 3. 요청 전송
        add_step("message_added", f"질문 전송: '{user_message[:30]}...' " if len(user_message) > 30 else f"질문 전송: '{user_message}'")
        
        # 스트리밍 응답 요청
        stream_response = openai_client.responses.create(
            stream=True,
            conversation=conversation.id,
            input=user_message,
            extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
        )
        complete_last_step("질문 전송 완료")
        
        # 4. 스트리밍 이벤트 처리
        for event in stream_response:
            # 응답 시작 - 웹 검색 시작으로 간주
            if event.type == "response.created":
                if not web_search_started:
                    add_step("tool_calls", "🔍 웹 검색 시작...")
                    web_search_started = True
            
            # 웹 검색 함수 호출 감지
            elif event.type == "response.output_item.added":
                if hasattr(event, 'item'):
                    item_type = getattr(event.item, 'type', '')
                    if item_type == "function_call":
                        func_name = getattr(event.item, 'name', 'unknown')
                        if 'search' in func_name.lower() or 'web' in func_name.lower():
                            complete_last_step("🔍 웹 검색 시작")
                            add_step("tool_calls", "🌐 검색 쿼리 실행 중...")
            
            # 함수 호출 완료 (웹 검색 결과 수신)
            elif event.type == "response.output_item.done":
                if hasattr(event, 'item'):
                    item_type = getattr(event.item, 'type', '')
                    
                    # 함수 호출 완료 = 웹 검색 결과 수신
                    if item_type == "function_call":
                        complete_last_step("🌐 웹 검색 완료")
                        add_step("file_search", "📄 검색 결과 분석 중...")
                    
                    # 함수 호출 결과 처리
                    elif item_type == "function_call_output":
                        complete_last_step("📄 검색 결과 분석 완료")
                        add_step("message_creation", "✍️ 응답 생성 중...")
                    
                    # 메시지 완료 - URL 인용 수집
                    elif item_type == "message":
                        for content in event.item.content:
                            if content.type == "output_text" and content.annotations:
                                for annotation in content.annotations:
                                    if annotation.type == "url_citation":
                                        sources.append(annotation.url)
            
            # 텍스트 스트리밍 시작
            elif event.type == "response.output_text.delta":
                if not response_started:
                    # 웹 검색이 시작되지 않았다면 (직접 응답하는 경우)
                    if web_search_started and len(steps) > 0 and steps[-1]["type"] == "tool_calls":
                        complete_last_step("🌐 웹 검색 결과 수신 완료")
                        add_step("message_creation", "✍️ 응답 생성 중...")
                    elif not any(s["type"] == "message_creation" for s in steps):
                        add_step("message_creation", "✍️ 응답 생성 중...")
                    response_started = True
                
                full_response += event.delta
            
            # 응답 완료
            elif event.type == "response.completed":
                complete_last_step("✍️ 응답 생성 완료")
                add_step("completed", "✅ 모든 작업 완료!", status="completed")
        
        # 최종 결과 표시
        total_time = time.time() - start_time
        clear_output(wait=True)
        display_steps(steps, title="🌐 WebSearch Agent 완료", is_complete=True)
        display_final_response(full_response, len(steps), total_time)
        
        # 출처 표시
        if show_sources and sources:
            print("\n📚 참고한 출처:")
            for i, url in enumerate(sources, 1):
                print(f"   {i}. 🔗 {url}")
        
        # Conversation 정리
        openai_client.conversations.delete(conversation_id=conversation.id)
        
        return full_response
        
    except Exception as e:
        add_step("error", f"오류 발생: {str(e)}", status="error")
        raise


print("✅ run_websearch_agent_with_steps() 함수 정의 완료!")
print("   사용법: run_websearch_agent_with_steps('오늘 서울 날씨는?')")

✅ run_websearch_agent_with_steps() 함수 정의 완료!
   사용법: run_websearch_agent_with_steps('오늘 서울 날씨는?')


In [33]:
# 상세 단계별 스트리밍 실행 예제
response = run_websearch_agent_with_steps("2024년 가장 인기있는 프로그래밍 언어는?")

2024년 가장 인기 있는 프로그래밍 언어는 여러 글로벌 순위(TIOBE Index, PYPL, Stack Overflow 등)를 종합해볼 때 다음과 같습니다.

**TIOBE Index(2024년 12월 기준) TOP 10**
1. 파이썬 (Python)  
2. C++  
3. 자바 (Java)  
4. C  
5. C#  
6. 자바스크립트 (JavaScript)  
7. Go  
8. SQL  
9. 비주얼 베이직 (Visual Basic)  
10. 포트란 (Fortran)  
[ITWorld](https://www.itworld.co.kr/article/3623944/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%ED%8B%B0%EC%98%A4%EB%B2%A0-%EC%84%A0%EC%A0%95-2024%EB%85%84-%EC%98%AC%ED%95%B4%EC%9D%98-%EC%96%B8%EC%96%B4%EB%A1%9C-%EC%9C%A0%EB%A0%A5.html), [네이버 블로그](https://blog.naver.com/PostView.naver?blogId=bestaicoach&logNo=223791927303)

**PYPL Index(2024년 12월 기준) TOP 10**
1. 파이썬 (Python)
2. 자바 (Java)
3. 자바스크립트 (JavaScript)
4. C/C++
5. C#
6. R
7. PHP
8. 타입스크립트 (TypeScript)
9. 러스트 (Rust)
10. 오브젝트-C (Objective-C)
[ITWorld](https://www.itworld.co.kr/article/3623944/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%ED%8B%B0%EC%98%A4%EB%B2%A0-%EC%84%A0%EC%A0%95-2024%EB%85%84-%EC%98%AC%ED%95%B4%EC%9D%98-%EC%96%B8%EC%96%B4%EB%A1%9C-%EC%9C%A0%EB%A0%A5.html)

**Stack Overflow Developer Survey (2023년 기준, 2024 트렌드 반영)**
- JavaScript, HTML/CSS, Python, SQL, TypeScript, Java, Bash/Shell, C#, C++ 등의 언어들이 많이 사용됨.
[Stack Overflow Developer Survey](https://survey.stackoverflow.co/2023/#section-most-popular-technologies-programming-scripting-and-markup-languages)

**요약 및 주요 트렌드**
- 데이터 분석, 인공지능, 자동화 분야의 확장으로 Python의 압도적 상승세가 이어졌습니다.
- 웹 개발에서는 JavaScript와 TypeScript가 계속 강세입니다.
- 시스템/게임/임베디드 분야에서는 C++, C, 그리고 Rust의 성장이 주목받고 있습니다.
- 백엔드와 클라우드 분야에서는 Go와 SQL의 채용률이 높아지고 있습니다.

**특히 많이 언급되는 언어**
- Python, JavaScript, Java, C++, C#, Go, Rust, TypeScript, SQL, PHP 등이 2024년 트렌드의 중심입니다.

---
더 자세한 자료와 해설은 아래 링크에서 확인 가능합니다:
- [ITWorld: 2024년 티오베 선정 인기 언어](https://www.itworld.co.kr/article/3623944/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%ED%8B%B0%EC%98%A4%EB%B2%A0-%EC%84%A0%EC%A0%95-2024%EB%85%84-%EC%98%AC%ED%95%B4%EC%9D%98-%EC%96%B8%EC%96%B4%EB%A1%9C-%EC%9C%A0%EB%A0%A5.html)
- [2024-2025 TIOBE Index 비교](https://blog.naver.com/PostView.naver?blogId=bestaicoach&logNo=223791927303)
- [PYPL Popularity of Programming Languages](https://pypl.github.io/PYPL.html)


📚 참고한 출처:
   1. 🔗 https://blog.naver.com/PostView.naver?blogId=bestaicoach&logNo=223791927303
   2. 🔗 https://www.itworld.co.kr/article/3623944/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%ED%8B%B0%EC%98%A4%EB%B2%A0-%EC%84%A0%EC%A0%95-2024%EB%85%84-%EC%98%AC%ED%95%B4%EC%9D%98-%EC%96%B8%EC%96%B4%EB%A1%9C-%EC%9C%A0%EB%A0%A5.html
   3. 🔗 https://survey.stackoverflow.co/2023/#section-most-popular-technologies-programming-scripting-and-markup-languages
   4. 🔗 https://pypl.github.io/PYPL.html


## 📋 요약

### 제공 함수

| 함수 | 설명 |
|------|------|
| `run_websearch_agent()` | 일반 호출 방식 (결과만 표시) |
| `run_websearch_agent_with_steps()` | 스트리밍 + 단계별 진행 상황 시각화 |

---

### 에이전트 작업 흐름

```
┌─────────────────────────────────────────────────────────┐
│  1️⃣  OpenAI Client 준비 완료                            │
│  2️⃣  Conversation 생성 완료                             │
│  3️⃣  질문 전송 완료                                     │
│  4️⃣  🔍 웹 검색 시작                                    │
│  5️⃣  🌐 웹 검색 완료                                    │
│  6️⃣  📄 검색 결과 분석 완료                             │
│  7️⃣  ✍️  응답 생성 완료                                 │
│  8️⃣  ✅ 모든 작업 완료!                                 │
└─────────────────────────────────────────────────────────┘
```

---

### 스트리밍 이벤트

| 이벤트 | 설명 |
|--------|------|
| `response.created` | 응답 시작 |
| `response.output_item.added` | 함수 호출 시작 (웹 검색 등) |
| `response.output_item.done` | 함수 호출 완료 / 메시지 완료 |
| `response.output_text.delta` | 응답 텍스트 토큰 스트리밍 |
| `response.completed` | 모든 작업 완료 |

---

### 핵심 API

```python
# 1. 클라이언트 설정
client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)
openai_client = client.get_openai_client()

# 2. WebSearch 에이전트 생성
definition = PromptAgentDefinition(
    model="gpt-4.1",
    instructions="...",
    tools=[WebSearchPreviewTool()]
)
agent = client.agents.create(name="WebSearchAgent", definition=definition)

# 3. Conversation 생성
conversation = openai_client.conversations.create()

# 4. 스트리밍 호출
stream = openai_client.responses.create(
    stream=True,
    conversation=conversation.id,
    input="질문",
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}}
)

# 5. 이벤트 처리
for event in stream:
    match event.type:
        case "response.created":
            print("🔍 웹 검색 시작...")
        case "response.output_item.done":
            print("🌐 검색 완료")
        case "response.output_text.delta":
            print(event.delta, end="", flush=True)
        case "response.completed":
            print("✅ 완료!")
```

---

**다음 노트북** → Semantic Kernel 멀티 에이전트 워크플로우